# Problems covered

1. Search for a number in a sorted, but rotated array.
2. Generate all unique letter combinations given a string.
3. Generate all possible valid sequences with $n$ pairs of parenthesis.
4. Shortest path between two words -- all intermediate words should appear in the dict.
5. Minimum number of meeting rooms, given meeting internvals.
6. Dictionary Word Break -- can you break a large word into sub-words from a dict?
7. LCA of two nodes in a binary tree.
8. Implement a `GetRandom` set

## 1. Search for a number in a sorted, but rotated array. 

The array could have been rotated `n` times.

* Original: [1, 2, 3, 4, 5, 6]
* Rotated: [5, 6, 1, 2, 3, 4]

### Questions

Linear scan is simple. Can you do better?

Because array is rotated, there is an inflection point: [5, **6**, 1, 2, 3, 4] : all elements upto this point are sorted, and after this point are also sorted.

Say you are searching for `5`. When you start, `i=0, j=5`.
* `m` will be `2` and `array[m] = 1`
* If `m` value is larger than `i` value, then array from `i to m` is sorted:
    * And if target is within array values from `[i, m)` set `j = m-1`; else `i = m+1`
* Similarly, if `m` value is less than `i` value, then array from `m to j` is sorted:
    * And if target is within array values from `(m, j]` set `i = m+1`; else `j = m-1`

In [11]:
class SearchInSortedRotated:
    def search(self, array, n):
        if len(array) == 0:
            return None
        if len(array) == 1:
            return 0 if array[0] == n else None
        
        i = 0
        j = len(array)-1
        
        while i <= j:
            m = i + (j - i) // 2
            if array[m] == n:
                return m
            else:
                if array[m] >= array[i]:
                    # Sorted upto m
                    if n >= array[i] and n < array[m]:
                        j = m - 1
                    else:
                        i = m + 1
                else:
                    if n > array[m] and n <= array[j]:
                        i = m + 1
                    else:
                        j = m - 1
                        
        return None


def search_in_sorted_rotated():
    s = SearchInSortedRotated()
    assert s.search([5, 6, 1, 2, 3, 4], 5) == 0
    assert s.search([5, 6, 1], 1) == 2
    assert s.search([5, 6], 6) == 1
    assert s.search([5, 9], 10) == None
    
search_in_sorted_rotated()

## 2. All unique non-empty letter combinations from a string.

Given string `AABC`, generate all non-empty string combinations (EXCLUDE ARRANGEMENTS).
* `A B C`
* `AA AB AC BC`
* `AAB AAC ABC`
* `AABC`

### Questions
1. How would you generate all FIXED LENGTH strings?

* Start with first letter, then include second letter -- now you have two letters `AA`
    * Now, you should remove the *last* letter, and proceed to the next letter ==> `AB`
        * Again, you have two letters, so you remove the last and go to next letter ==> `AC`
            * Now the sequence has ended, so you stop.
            
2. Can you improve this?? Between 2-len and 3-len combinations, the substrings will repeat. 
* So, instead of waiting for the length to equal $n$, you can add the current substring.

Complexity: exponential. Time complexity  $nCk$ is $o(n^k)$ . But if you are generating all combinations (all unique subsets), that is $o(2^n)$

In [1]:
class UniqueLetterCombinations:
    def solve(self, string):
        ans = set()
        
        def combinations_fixed(current, n, pick):
            if len(current) == n:
                ans.add(''.join(current))
            else:
                for i in range(pick, len(string)):
                    current.append(string[i])
                    combinations_fixed(current, n, i+1)
                    current.pop()
                    
        for size in range(1, len(string)+1):
            combinations_fixed([], size, 0)
            
        return ans
    
    
    def solve_direct(self, string):
        """
        This solution does not create combinations of fixed size.
        Rather, it considers all lengths, as it is progressing.
        """
        ans = set()
        def progressive(current, pick):
            for i in range(pick, len(string)):
                current.append(string[i])
                temp = ''.join(current)
                ans.add(temp)
                
                progressive(current, i+1)
                current.pop()
                
        progressive([], 0)
        return ans
                
    
def unique_letter_combinations():
    o = UniqueLetterCombinations()
    o.solve('ABCDEFGHIJKLMNOPQRST')
    
unique_letter_combinations()

In [2]:
o = UniqueLetterCombinations()

In [3]:
%timeit o.solve('ABCDEFGHIJKLMNOP')

432 ms ± 9.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit o.solve_direct('ABCDEFGHIJKLMNOP')

73.3 ms ± 919 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 3. All valid parenthesis with $n$ pairs of parenthesis

If n == 2: `()(), (())`

If n == 3: `()()(), ()(()), ((())), (())()`

### Questions

Simplest solution is to brute force.
* `(` + all valid sequences with `n-1` parenthesis
* `)` + all valid sequences with `n-1` parenthesis

Complexity will be $2^{2N}N$ -- the final $N$ is because we use `join` on the list.

**Another** solution, is to prune the number of bad solutions:
* Only add an open-brace when available and closed are also available.
* Only add closed brace when closed is available AND number of closed < number of open.

Time complexity of this solution will be a lot smaller -- because you are not considering all possible combinations -- but exact answer ...?

In [24]:
class FixedLenParenthesis:
    def solve(self, n):
        ans = []
        
        def isValid(seq):
            net = 0
            for c in seq:
                if net < 0:
                    return False
                
                if c == '(':
                    net += 1
                else:
                    net -= 1
                    
            return net == 0

        
        def combine(current):
            if len(current) == 2*n:
                if isValid(current):
                    ans.append(''.join(current))
                    
            else:
                current.append('(')
                combine(current)
                current.pop()
                
                current.append(')')
                combine(current)
                current.pop()
                
        combine([])
        return ans
    
    
    def pruning(self, n):
        """
        In this version, we keep checking on the number of
        open and closed braces used.
        
        Rules:
            Only add an open-brace when available and closed are also available
            Only add a closed-brace when available, and does not exceed open
        """
        ans = []
        
        def aware(current, br_open, br_close):
            if len(current) == 2*n:
                ans.append(''.join(current))
            else:
                if br_open < n and br_close < n:
                    current.append('(')
                    aware(current, br_open+1, br_close)
                    current.pop()
                    
                if br_close < n and br_close < br_open:
                    current.append(')')
                    aware(current, br_open, br_close+1)
                    current.pop()
                    
        aware([], 0, 0)
        return ans

In [25]:
o = FixedLenParenthesis()
o.solve(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

In [26]:
o.pruning(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

In [28]:
%timeit o.solve(8)

87.8 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%timeit o.pruning(8)

3.62 ms ± 32.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 4. Shortest path between two words -- all intermediate words should appear in the dict.

*The next word is generated by substituting a letter from the previous word.* 

Start word: `hit`

End word: `cog`

Dictionary: `hit, cog, hog, hot, bot, cot`

Shortest path: `hit -> hot -> cot -> cog`

### Questions

Since question is asking for shortest path, **BFS should come to mind!** During BFS, whenever you find the target word, just return it (along with the length so far).

**Complexity**: Consider $N$ words in the list, and length of each word is $L$. Worst-case scenario:
* we start from the first word, and iterate over all its letters => $O(L)$
    * for each of these letters, we generate the pattern, which takes $O(L)$
        * and we iterate over all words for each pattern, which could be $O(N)$
* Thus, total complexity: O(L x L x N) => $O(L^2.N)$

In [53]:
from collections import deque
from collections import defaultdict
import string

class ShortestPahtBetweenWords:
    def solve(self, start, end, words):
        """
        Words is a list.
        Start and End are words.
        """
        words = set(words)
        
        if start not in words or end not in words:
            return -1
        if start == end:
            return 0
        
        seen = set()

        que = deque([(start, 0)])
        
        while que:
            current, d = que.popleft()
            if current == end:
                return d
            
            if current in seen:
                continue
                
            seen.add(current)
            for i in range(len(start)):
                pre = current[:i]
                aft = current[i+1:]
                for char in string.ascii_lowercase:
                    temp = pre + char + aft
                    if temp in words and temp not in seen:
                        que.append((temp, d+1))
                        
        return -1
    
    def smarter(self, start, end, words):
        words = set(words)
        if start not in words or end not in words:
            return -1
        if start == end:
            return 0
        
        # word: "bat" ; patterns: "_at", "b_t", "ba_"
        possible = defaultdict(set)
        for w in words:
            for i in range(len(start)):
                pre = w[:i]
                aft = w[i+1:]
                pattern = pre + '_' + aft
                possible[pattern].add(w)
        
        que = deque([(start, 0)])
        seen = set()
        while que:
            current, d = que.popleft()
            if current == end:
                return d
            
            if current in seen:
                continue
                
            seen.add(current)
            for i in range(len(start)):
                pre = current[:i]
                aft = current[i+1:]
                pattern = pre + '_' + aft
                for temp in possible[pattern]:
                    if temp not in seen:
                        que.append((temp, d+1))
                        
        return -1

In [54]:
o = ShortestPahtBetweenWords()

In [45]:
%timeit o.solve('hit', 'lie', ['hit', 'cog', 'bog', 'bat', 'bit', 'sit', 'kit', 'hot', 'cot', 'dit', 'dog', 'dig', 'bye', 'lie', 'dye', 'die'])

156 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [55]:
%timeit o.smarter('hit', 'lie', ['hit', 'cog', 'bog', 'bat', 'bit', 'sit', 'kit', 'hot', 'cot', 'dit', 'dog', 'dig', 'bye', 'lie', 'dye', 'die'])

67.1 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 5. Find minimum number of meeting rooms for the schedule.

Given meeting `(start, end)` times: `[(0, 30), (5, 15), (20, 50), (10, 100)]`

Can you find the minimum number of meeting rooms for these meetings?

### Questions

1. A simple way to solve this, is to sort everything by meeting-start-time
    * Then you look at the first meeting, assign it a room,
    * and check if the next meeting starts before the first one ends,
        * If yes, then that same room gets used.
        * Else, assign a new meeting room.
2. In this solution, you will have to search for the meeting room that is available at the earliest.
3. Complexity will be $O(N^2)$ with num of meetings -- worst case, meetings start very close to each other, and we search over all rooms for every meeting.

**A faster way** would be to always track the EARLIEST AVAILABLE MEETING ROOM!
1. You can use a `min-heap` for this.
2. Add first-meeting's end time to `heap`.
3. For the next meeting, check if start time is BEFORE end time.
    * If True, then you need a new meeting room; run `heappush` to add new end time.
    * If False, then same meeting room can be used, and it's end-time will have to be udpated.
        * Do `heapq.heapreplace` => this will pop the smallest value and `heappush` the new end time.
4. Complexity will be:
    * N.log(N) for the sort, plus
    * Iterating over each meeting => N
        * and at each meeting running `heappush` => log(N)

Total => O(N.log(N) + N.log(N))

In [77]:
import heapq

class MinimumMeetingRooms:
    def solve(self, meetings):
        """
        meetings: List of meetings with start and end times.
        """
        meetings.sort(key=lambda t:t[0])
        
        rooms = {}  # room_id => end_time
        
        for start, end in meetings:
            found = False
            for r, available in rooms.items():
                if start >= available:
                    found = True
                    rooms[r] = end
                    break
                    
            if found is False:
                rooms[len(rooms)] = end
                
        return len(rooms)
    
    
    def alwaysAvailable(self, meetings):
        """
        Uses a minheap to track the earliest available rooms.
        """
        meetings.sort(key=lambda t: t[0])
        
        if len(meetings) <= 1:
            return len(meetings)
        
        rooms = []
        rooms.append(meetings[0][1])
        
        for i in range(1, len(meetings)):
            start, end = meetings[i]
            earliest = rooms[0]
            if start < earliest:
                # You will need a new room.
                heapq.heappush(rooms, end)
            else:
                heapq.heapreplace(rooms, end)
                
        return len(rooms)

In [78]:
o = MinimumMeetingRooms()

In [79]:
o.solve([(0, 30), (5, 15), (20, 50), (10, 100)])

3

In [80]:
o.alwaysAvailable([(0, 30), (5, 15), (20, 50), (10, 100)])

3

## 6. LRU cache

**implement PUT**
1. Add (k, v) pair if not exists, or update value if it exists.
2. If `cache` is at capacity, remove the LRU key and add the new one.

**implement GET**
1. Return value if key is present; else -1

The main complexity is to constantly move the most recently used element (via `put` or `get`) at the end of the structure.

The simple method is to use the OrderedDict -- there is a push to end function, which will move the key to then end after the `get` call. This way, the `first` element of the Dict will always be the `LRU` element. This method also gives O(1) for both operations.

Quite possible the interviewer will not entertain this -- the alt approach is to implement a **Doubly Linked List**.
* For each get/put on an existing key, you move that node to the end
* All new keys are added to the end
* A separate dictionary (cache) maps the keys to the DLL nodes

In [125]:
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()
        
    def get(self, key):
        if key in self.cache:
            self.cache.move_to_end(key)
            return self.cache[key]
        else:
            return -1
        
    def put(self, key, value):
        if key in self.cache:
            self.cache[key] = value
            self.cache.move_to_end(key)
        else:
            if len(self.cache) == self.capacity:
                remove = next(iter(self.cache))
                del self.cache[remove]
                
            self.cache[key] = value
            
            

class DLLNode:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.before = None
        self.after = None
        
        
class DLLCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = {}  # key => Node(key, val)
        
        self.head = None
        self.tail = None
        
    def addNewNode(self, node):
        if len(self.cache) == 0:
            self.head = node
            self.tail = node
            self.cache[node.key] = node
        else:
            assert self.head is not None
            assert self.tail is not None
            
            self.tail.after = node
            node.before = self.tail
            self.tail = node
            self.cache[node.key] = node
        
    def moveToEnd(self, node):
        if node.before is None and node.after is None:
            pass  # Single element
        else:
            before = node.before
            after = node.after
            
            # Now put this after the tail.
            self.tail.after = node
            node.before = self.tail
            node.after = None
            self.tail = node
            
            # Now bridge the gap.
            after.before = before
            if before is not None:
                before.after = after
            else:
                self.head = after
        
    def get(self, key):
        if key in self.cache:
            # Move to end,
            # then return.
            node = self.cache[key]
            self.moveToEnd(node)
            return self.cache[key].val
        else:
            return -1
        
    def put(self, key, val):
        if key is self.cache:
            node = self.cache[key]
            node.val = val
            self.moveToEnd(node)
        else:
            if len(self.cache) == self.capacity:
                # Remove the HEAD element.
                if len(self.cache) == 1:
                    self.cache = {}
                    self.head = None
                    self.tail = None
                else:
                    head = self.head
                    after = head.after
                    
                    head.after = None
                    after.before = None
                    del self.cache[head.key]
                    
                    self.head = after
                    assert self.head.before is None
                    
            self.addNewNode(DLLNode(key, val))

In [126]:
o = LRUCache(3)
assert o.get(1) == -1
o.put(1, 1)
assert o.get(1) == 1
o.put(2, 2)
o.put(3, 3)

o.put(1, 5)
o.put(4, 4)

assert o.get(1) == 5
assert o.get(2) == -1
assert o.get(3) == 3

In [127]:
o = DLLCache(3)
assert o.get(1) == -1
o.put(1, 1)
assert o.get(1) == 1
o.put(2, 2)
o.put(3, 3)

o.put(1, 5)
o.put(4, 4)

assert o.get(1) == 5
assert o.get(2) == -1
assert o.get(3) == 3

## 6. Dictionary Word Break -- can you confirm if a large word can be broken into sub-words from a dict?

* string `applebananaapple`
* dict `[apple, app, ban, banana]`

You can begin by brute-forcing it ... that's not a good idea.

Maybe do it progressively?
1. Start with original word, check if it's in the dict; return True.
2. If not, iterate over every position => O(L)
    * Create suffix and prefix => O(L)
    * And check for prefix PRESENT and recursively start from Step1. with the suffix.
   
$O(n^3)$

In [178]:
class DictionaryWordBreak:
    def solve(self, word, dictionary):
        """
        Usually, `dictionary` will be a list ...
        CONVERT TO SET!
        """
        dictionary = set(dictionary)
        
        if word in dictionary:
            return [word]
        
        if len(dictionary) == 0:
            return []
        if len(dictionary) == 1:
            return [word] if next(iter(dictionary)) == word else []
        
        seen = {}
        def check(token):
            if token in seen:
                return seen[token]
            
            if token in dictionary:
                seen[token] = True
                return True
            
            for j in range(len(token)):
                prefix = token[:j]
                suffix = token[j:]
                if prefix in dictionary and check(suffix):
                    seen[token] = True
                    return True
                
            seen[token] = True
            return False
        
        return check(word)

In [179]:
o = DictionaryWordBreak()

In [180]:
o.solve('applebananaapplebees', ['apple', 'app', 'ban', 'banana', 'pen', 'leb', 'bees'])

True

## 7. LCA of two nodes in a binary tree.

A `common ancestor` is a node for which both target `p` and `q` are its children. A node also counts as its own ancestor.

LCA is the `lowest` common ancestor (aka the `deepest`, because the `root` will always be a common ancestor...)

### Questions

A generic solution will be to search first for the lowest CA, and then propagate this information upwards, to get all ancestors
* Start traversing the tree in a DFS manner (Left only, then right).
* If you find p, return that `p` has been `found`; similarly if you find `q` return that `q` has been found.
* Now, check at the current node is LCA conditions are met:
    * Current node is `p` or `q`
    * Children contain `p` or `q`
    * Return this information upwards (either `p` is found, or `q` is found)
    
**Complexity** is $O(N)$ where $N$ is total nodes -- each node is visited once during the DFS traversal.

In [186]:
class Node:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        
        
class LCATwoNodes:
    def solve(self, root, p, q):
        assert isinstance(root, Node)
        assert isinstance(p, Node)
        assert isinstance(q, Node)
        
        self.ans = None
        
        def find(n, found_p, found_q):
            if n is None:
                return False, False
            else:
                left_p, left_q = find(n.left, found_p, found_q)
                right_p, right_q = find(n.right, found_p, found_q)
                
                ret_p = left_p or right_p or n.val == p.val
                ret_q = left_q or right_q or n.val == q.val
                
                if ret_p and ret_q and self.ans is None:
                    self.ans = n
                    
                return ret_p, ret_q
            
        find(root, False, False)
        
        return self.ans

In [189]:
root = Node(0)
one = Node(1)
two = Node(2)
three = Node(3)
four = Node(4)
five = Node(5)
six = Node(6)
seven = Node(7)
eight = Node(8)

root.left = one
root.right = two
one.left = three
three.right = four
one.right = five
five.right = six
six.left = seven
three.left = eight

o = LCATwoNodes()
assert o.solve(root, three, four) == three
assert o.solve(root, eight, four) == three
assert o.solve(root, two, seven) == root

## 8. Implement a `GetRandom` set

Implement all of the following, with **O(1) complexity each**:
* `put(x)` : add x to set if not present
* `get(x)` : True if x is in set
* `del(x)` : Drop if x is in set and return True; else return False
* `sample()` : Return an element from the set at random.